# Лабораторная работа №10 “Градиентный бустинг”


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
from scipy import misc
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
import copy
from matplotlib import cm
from matplotlib.animation import FuncAnimation
import scipy.optimize
import os
from sklearn import svm
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist
from scipy import stats
from sklearn.tree import *
from sklearn.ensemble import *


import math
from sklearn.model_selection import train_test_split

## 1. Загрузите данные с помощью библиотеки sklearn.


In [11]:
from sklearn.datasets import load_boston
boston_dataset = load_boston()
x = boston_dataset["data"]
y = boston_dataset["target"]

## 2. Разделите выборку на обучающую (75%) и контрольную (25%).


In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=.75, test_size=.25, random_state=42)
features = boston_dataset.feature_names

## 3. Заведите массив для объектов DecisionTreeRegressor (они будут использоваться в качестве базовых алгоритмов) и для вещественных чисел (коэффициенты перед базовыми алгоритмами).

In [14]:
decision_trees = []
alphas = []
tree_count = 50
max_depth = 5
random_state = 42

## 5. В цикле обучите последовательно 50 решающих деревьев с параметрами max_depth=5 и random_state=42 (остальные параметры - по умолчанию). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с отклонением истинных значений от предсказанных.

In [16]:
y_shift = y_train.copy()
for i in range(tree_count):
    regressor = DecisionTreeRegressor(random_state=random_state, max_depth=max_depth)
    regressor.fit(x_train, y_shift)
    y_shift -= y_shift - regressor.predict(x_train)
    decision_trees.append(regressor)

## 6. Попробуйте всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но на стандартном наборе данных будет всего 50 деревьев, возьмите для начала шаг побольше.


In [17]:
decision_trees_v2 = []

y_shift = y_train.copy()
for i in range(tree_count):
    regressor = DecisionTreeRegressor(random_state=random_state, max_depth=max_depth)
    regressor.fit(x_train, y_shift)
    y_shift -= (y_shift - 0.9*regressor.predict(x_train))
    decision_trees_v2.append(regressor)

## 7. В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке X. Реализуйте ее. Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью mean_squared_error в sklearn.metrics.


In [10]:
def rmse(predictions, targets):
    differences = predictions - targets                       
    differences_squared = differences ** 2                    
    mean_of_differences_squared = differences_squared.mean()  
    rmse_val = np.sqrt(mean_of_differences_squared)           
    return rmse_val                                           